In [1]:
import os
import requests
import json
from PIL import Image

In [2]:
image_path = ".\\output\\Khanh_HN_pages_2\\"
image_name = "Khanh_TCQN_HN_page_"
image_name_2 = "Khanh_TNYL_P2_HN_page"
output_path = ".\\output\\OCR_HN\\"
output_name_1 = "ocr_"
output_name_2 = "ocr_3_"
upload_url = r"https://tools.clc.hcmus.edu.vn/api/web/clc-sinonom/image-upload"
ocr_url = r"https://tools.clc.hcmus.edu.vn/api/web/clc-sinonom/image-ocr"

In [3]:
headers = {"User-Agent":"test 123"}

In [6]:
def ocr_image_with_clc_api(image_name, index, output_name):
    with open(os.path.join(image_path, image_name), "rb") as image_file:
        files = {
            "image_file": image_file
        }
        upload_response = requests.post(upload_url, files=files,headers=headers)
    upload_response=upload_response.json()
    print(upload_response)
    body = {
        "file_name": upload_response.get('data').get('file_name'),
        "ocr_id": 1
    }
    
    ocr_response = requests.post(ocr_url, json=body,headers=headers)
    if ocr_response.status_code == 200:
        try:
            result = ocr_response.json()
        except ValueError:
            print(f"Response từ server không hợp lệ cho ảnh {image_name}.")
            return

        if result.get("is_success"):
            print(f"OCR ảnh {image_name} thành công!")
            transcription = result["data"].get("result_ocr_text", [])
            points = result["data"].get("result_bbox", [])

            listOfbox = [{"transcription": transcription[i], "points": points[i]} for i in range(len(points))]
            image = Image.open(os.path.join(image_path, image_name))
            data = {
                "image_name": image_name,
                "OCR_Result": listOfbox,
                "width_image": image.size[0],
                "height_image": image.size[1],
            }

            if not os.path.exists(output_path):
                os.makedirs(output_path)

            Output_Filename = f"{output_name}{index}.json"
            with open(os.path.join(output_path, Output_Filename), "w", encoding="utf-8") as f:
                json.dump(data, f, ensure_ascii=False, indent=4)

        else:
            print(f"Lỗi OCR ảnh {index}: {result['code']}, {result.get('message', 'Không có thông tin lỗi')}")
    else:
        print(f"Request thất bại với mã trạng thái: {ocr_response.get('code')}")
        print(ocr_response.get('message'))

In [5]:
start = 1
end = 22

In [7]:
for index in range(start, end + 1):
    image_name = image_name_2+ str(index).zfill(3) + ".png"
    ocr_image_with_clc_api(image_name, index, output_name_2)

{'is_success': True, 'code': '000000', 'message': None, 'data': {'file_name': 'temp628ffd1148e94ac8a9e22e1c8eedb5f2.png'}}
OCR ảnh Khanh_TNYL_P2_HN_page001.png thành công!
{'is_success': True, 'code': '000000', 'message': None, 'data': {'file_name': 'temp70c55c66f8a74cc6ac0f9c74d444834f.png'}}
OCR ảnh Khanh_TNYL_P2_HN_page002.png thành công!
{'is_success': True, 'code': '000000', 'message': None, 'data': {'file_name': 'temp0b8aca0f6d4445ae83297cff18f44b80.png'}}
OCR ảnh Khanh_TNYL_P2_HN_page003.png thành công!
{'is_success': True, 'code': '000000', 'message': None, 'data': {'file_name': 'tempa1c61a8838bd47959601df8a92366d19.png'}}
OCR ảnh Khanh_TNYL_P2_HN_page004.png thành công!
{'is_success': True, 'code': '000000', 'message': None, 'data': {'file_name': 'tempe704ba61a77c43eab9c7d3949e38fc80.png'}}
OCR ảnh Khanh_TNYL_P2_HN_page005.png thành công!
{'is_success': True, 'code': '000000', 'message': None, 'data': {'file_name': 'temp63259ca33cc64f1680a81b6172564227.png'}}
OCR ảnh Khanh_TNY

# **Clean data**

In [7]:
def cleanup_OCR_File1(data_path, image_name, output_path):
    with open(os.path.join(data_path, image_name), "r", encoding="utf-8") as file:
        data = json.load(file)
    
    OCR_result_list = data['OCR_Result']
    item = OCR_result_list[0]
    while(len(item['transcription']) == 1):
        del OCR_result_list[0]
        item = OCR_result_list[0]
    if(len(OCR_result_list) < 9):
        print(f"file {image_name} deo on roi sep oi")
    data['OCR_Result'] = OCR_result_list

    Output_Filename = f"ocr_3_Cleaned_{index}.json"
    with open(os.path.join(output_path, Output_Filename), "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [8]:
for index in range(1, 22 + 1):
    # if(index == 49 or index == 60):
    #     continue
    image_name = "ocr_3_" + str(index) + ".json"
    cleanup_OCR_File1(".\\output\\OCR_HN\\", image_name, ".\\output\\OCR_Cleaned_HN\\")

In [15]:
def cleanup_OCR_File2(data_path, image_name, output_path):
    with open(os.path.join(data_path, image_name), "r", encoding="utf-8") as file:
        data = json.load(file)
    
    OCR_result_list = data['OCR_Result']
    item = OCR_result_list[0]
    while(len(item['transcription']) <= 2):
        del OCR_result_list[0]
        item = OCR_result_list[0]
    if(len(OCR_result_list) < 9):
        print(f"file {image_name} deo on roi sep oi")
    data['OCR_Result'] = OCR_result_list

    Output_Filename = f"ocr_2_Cleaned_{index}.json"
    with open(os.path.join(output_path, Output_Filename), "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

In [16]:
for index in range(1, 25 + 1):
    image_name = "ocr_2_" + str(index) + ".json"
    cleanup_OCR_File2(".\\output\\OCR_HN\\", image_name, ".\\output\\OCR_Cleaned_HN\\")